# Análisis OLAP

El almacén de datos Hidro-Meto contiene observaciones de precipitación y temperatura de algunas estaciones ubicadas en distintas regiones del país.

El archivo `dw_hidro_meteo.sql` contiene el modelo del almacén de datos y los datos para poblarlo.

**Actividades**

1. Materializar un cubo en MySQL
2. En MySQL diseñar y ejecutar operaciones OLAP.
3. Conectar desde Python (`mysql-connector-python`), extraer y construir un cubo (agregaciones multidimensionales).
4. En Python diseñar y ejecutar operaciones OLAP.


**Requisitos**

- MySQL 8.x
- Python 3.9+
- Pandas
- mysql-connector-python


## 1. Construcción y exploración del cubo en MySQL

1. Crear el cubo (tabla)
2. Extraer los datos desde el DW y cargarlos en el cubo (tabla)
2. Explorar el cubo con operaciones OLAP

## 2. Extraer datos del cubo y exploración OLAP con Python

In [10]:
import src.dw_connection as dw

# Verificar conexión y versión de MySQL
cnx = dw.conexion(host='localhost', user='root', pwd='mysqlroot', dbname='dw_hidro_meteo')
cnx.close()

MySQL version: 8.1.0
Conexión establecida con exito!


In [13]:
import pandas as pd

cnx = dw.conexion(host='localhost', user='root', pwd='mysqlroot', dbname='dw_hidro_meteo')
query = "SELECT * FROM mi_cubo"

# Alternativa A) Crear create_engine con sqlalchemy
# df = pd.read_sql(query, cnx)

# Alternativa B) Directamente con mysql-connector
cursor = cnx.cursor()
cursor.execute(query)
results = cursor.fetchall()
columns = [field[0] for field in cursor.description]

# columnas = []
# for campo in cursor.description:
#     columnas.append(campo[0])

cubo_df = pd.DataFrame(results, columns=columns)

cursor.close()
cnx.close()

cubo_df.head(20)

MySQL version: 8.1.0
Conexión establecida con exito!


,anio,mes,region,estacion,precip_mm,tmax_c
0,2023,1,Arica y Parinacota,ST001,883.900,21.574
1,2023,1,Biobío,ST006,1006.700,21.542
2,2023,1,Coquimbo,ST003,734.200,21.294
3,2023,1,Metropolitana,ST005,971.800,22.155
4,2023,1,Tarapacá,ST002,981.800,24.313
5,2023,1,Valparaíso,ST004,707.600,21.868
6,2023,2,Arica y Parinacota,ST001,941.100,22.546
7,2023,2,Biobío,ST006,868.800,24.279
8,2023,2,Coquimbo,ST003,885.900,21.500
9,2023,2,Metropolitana,ST005,990.900,21.264


In [14]:
# Roll-up: precipitación total por estación
cubo_df.groupby(by="estacion")["precip_mm"].sum()

estacion
ST001    22400.400
ST002    22348.800
ST003    21319.400
ST004    21931.300
ST005    21960.800
ST006    21987.700
Name: precip_mm, dtype: object

In [19]:
# Drill-down: precipitación día a día para ST001
cubo_df[cubo_df["estacion"] == "ST003"][["region","anio","mes", "precip_mm"]]

,region,anio,mes,precip_mm
2,Coquimbo,2023,1,734.200
8,Coquimbo,2023,2,885.900
14,Coquimbo,2023,3,861.100
20,Coquimbo,2023,4,874.400
26,Coquimbo,2023,5,829.200
32,Coquimbo,2023,6,997.000
38,Coquimbo,2023,7,1110.700
44,Coquimbo,2023,8,945.400
50,Coquimbo,2023,9,892.400
56,Coquimbo,2023,10,883.500


In [20]:
# Slice: datos solo de ST002
cubo_df[cubo_df["estacion"]=="ST002"]

,anio,mes,region,estacion,precip_mm,tmax_c
4,2023,1,Tarapacá,ST002,981.800,24.313
10,2023,2,Tarapacá,ST002,905.100,21.411
16,2023,3,Tarapacá,ST002,919.200,21.119
22,2023,4,Tarapacá,ST002,1025.200,21.143
28,2023,5,Tarapacá,ST002,1004.500,22.406
34,2023,6,Tarapacá,ST002,961.400,21.680
40,2023,7,Tarapacá,ST002,1084.300,24.997
46,2023,8,Tarapacá,ST002,907.600,23.381
52,2023,9,Tarapacá,ST002,805.400,24.080
58,2023,10,Tarapacá,ST002,858.200,22.587


In [21]:
# Dice: promedio mensual de temperaturas
cubo_df.groupby(by=["estacion", "mes"])["tmax_c"].mean()

estacion  mes
ST001     1      21.8515
          2       22.756
          3      21.5145
          4        22.17
          5       23.603
                  ...   
ST006     8       23.577
          9       23.507
          10      24.632
          11     22.1585
          12     21.6595
Name: tmax_c, Length: 72, dtype: object

In [24]:
# Pivot: días como filas, estaciones como columnas
pivot = cubo_df.pivot_table(index="mes", columns="estacion", values="precip_mm", aggfunc="sum")
print(pivot)

estacion     ST001     ST002     ST003     ST004     ST005     ST006
mes                                                                 
1         1609.500  2061.400  1859.700  1769.700  1829.000  1912.000
2         1851.000  1875.000  1691.900  1937.000  1861.000  1713.400
3         1824.300  1901.100  1604.700  1882.900  1876.200  1724.900
4         1805.900  1854.400  1747.700  1829.900  1669.200  1893.800
5         1760.300  2058.700  1617.200  1771.800  1888.300  1749.700
6         1895.500  1705.100  1890.200  1693.500  1818.800  1905.200
7         1815.200  1861.000  1880.400  1643.500  1813.400  1833.800
8         1962.100  1913.100  2066.600  1737.400  1737.000  1847.700
9         2126.300  1663.000  1701.100  1946.500  1880.300  1733.300
10        2045.100  1760.500  1729.000  1768.400  1881.000  2121.500
11        1950.600  1866.300  1590.600  1777.400  1781.300  1743.200
12        1754.600  1829.200  1940.300  2173.300  1925.300  1809.200


## 3. Resumen

A partir del ejemplo, DW vs. Cubo,
- **DW**: granularidad diaria por estación y variable.
- **Cubo**: agregaciones por combinaciones de dimensiones. Responde más rápido para análisis recurrentes.

## 4. Actividades

El almacén de datos `dw_hidro_meteo` contiene observaciones diarias de precipitación y temperatura máxima.
Las dimensiones incluyen:
- Tiempo (día, mes, trimestre, año)
- Estación (código, nombre)
- Localización (latitud, longitud, comuna, región)
- Variable, precipitación (PRECIP) y temperatura máxima (TMAX)

Se dispone de datos para 3 años (2022–2024) y 8 estaciones distribuidas en distintas regiones de Chile.

El objetivo es construir un cubo OLAP que permita explorar y realizar un análisis de la información desde distintas perspectivas.

#### Ejercicio 1. Crear y poblar el cubo

1. Construir en MySQL el cubo `meteo`, agregando los datos mensuales por (año, mes, región, estación) con las siguientes medidas:
- `precipitacion_mm = SUM(PRECIP)`
- `temperatura_c = AVG(TMAX)`
- `dias_de_lluvia = COUNT(DIAS CON PRECIP > 0)`

2. Insertar en el cubo las agregaciones de todas las estaciones para el periodo comprendido entre 2022 y 2023.

3. Entregar un archivo SQL con,
- Sentencias SQL para crear del cubo (`CREATE TABLE`).
- Sentencias SQL para poblar el almacen de datos (`INSERT INTO`).
- Sentencias SQL para verificar que los datos fueron cargados correctamente (`SELECT`).

#### Ejercicio 2: Análisis OLAP

Para cada problema identificar la(s) operacion(es) OLAP que permite resolver cada pregunta. Justificar.

1.	Calcular la precipitación total anual por región y estación para cada año del periodo. Identificar la estación más lluviosa en cada año.
2. A partir de los totales anuales, desglose los valores de 2023 en meses para la Estación Santiago. Identifique el mes que tuvo más días de lluvia.
3. Filtre el cubo para analizar la región del Biobío. Compare la temperatura promedio entre los años comprendidos en el periodo e identifique si se presentan cambios significativos.
4. Extraiga un subcubo con los meses de junio, julio y agosto (invierno) para todas las estaciones. Además, calcule para cada estación la precipitación total en invierno, la temperatura promedio de invierno y los días de lluvia en invierno. Explique si existes alguna relación entre las estaciones más lluviosas y las más frías.
5. Construya una tabla en la cual, las filas correspondan a los meses del año 2023, las columnas correspondas a las estaciones y las celdas contengan las precipitaciones. Identifique los meses con contrastes importantes de lluvia entre estaciones.
6. Genere un indicador de intensidad de lluvia: `intensidad = precipitacion_mm / días_de_lluvia`. Calcule este valor para cada estación en 2024 e identifique las estaciones que presentan lluvias menos frecuentes pero más intensas.
7. Considere que el cubo se debe ampliar a 20 años y a más estaciones del territorio chileno. ¿Qué jerarquías temporales y espaciales serían útiles para el análisis? ¿Es conveniente mantener un cubo materializado o calcularlo a demanda? Justifique su respuesta.